In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate,LLMChain
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [2]:
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
embeddings = OpenAIEmbeddings()

In [3]:
loader = TextLoader('video_transcription/final_file')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(docs)

In [4]:
from langchain.vectorstores import DeepLake
db = DeepLake(dataset_path="./my_deeplake/", embedding_function=embeddings, read_only=True)

./my_deeplake/ loaded successfully.



Deep Lake Dataset in ./my_deeplake/ already exists, loading from the storage


Dataset(path='./my_deeplake/', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (683, 1536)   None     None   
    ids      text     (683, 1)      str     None   
 metadata    json     (683, 1)      str     None   
   text      text     (683, 1)      str     None   


In [5]:
query = "Why is attention important in the attention model?"

In [6]:
ans = db.similarity_search(query,k=5)
context = ""
for i in range(5):
    context += ans[i].page_content
    context += "\n"

In [12]:
llm = OpenAI(temperature=0)
template="""Please use the following context and history to answer questions.
Current conversation: {history}
Context: {ans}
Question: {query}
Answer:"""

In [18]:
prompt = PromptTemplate(template=template, input_variables=["history","ans","query"])
llm_chain = ConversationChain(prompt=prompt, llm=llm, memory=ConversationBufferMemory())
llm_chain.predict(ans = context, query = query)

ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['history', 'ans', 'query'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)